In [ ]:
import numpy as np
import babypandas as bpd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from IPython.display import display, IFrame

def merging_animation():
    src="https://docs.google.com/presentation/d/e/2PACX-1vSk2FfJ4K_An_CQwcN_Yu5unpJckOZjVQDFqZ78ZTTMmowUsCQKKVnum0_m6TaiGquQ44E3FiS9g2Y4/embed?start=false&loop=false&delayms=60000"
    width=825
    height=500
    display(IFrame(src, width, height))

# Lecture 10 – Grouping with Subgroups, Merge

## DSC 10, Winter 2022

### Announcements

- Homework 3 is due **Saturday 1/29 at 11:59pm.**
- Lab 4 is due **Tuesday 2/1 at 11:59pm**.
- Project 1 will be released tomorrow and will be due **Saturday 2/12 at 11:59pm**.
- Logistics regarding in-person instruction are on the next slide.

### Logistics starting January 31st

You may have heard the news – in-person instruction on campus is starting on 1/31. Here's what will happen in DSC 10:

- **Lectures** will be delivered **in-person** in Center Hall 214, but you can also **participate live via Zoom** (same link as before).
    - Tutors will monitor the Zoom chat and let me know if there's anything to address.
    - Lectures will still be recorded and posted online afterwards.
- Same plan for **discussions** – **in-person** in Center Hall 105 but with a **Zoom** remote option, with recordings posted afterwards.
- Most **office hours** will stay remote, but some will move in-person.
    - The [Calendar](https://dsc10.com/calendar/) will be updated this weekend to specify which office hours are in-person and which are remote, and with instructions on how to access in-person office hours.
- **Exams** will be remote.

### Agenda

- Grouping with subgroups.
- Merging.

## Grouping with subgroups

### Our familiar NBA dataset 🏀

In [ ]:
nba = bpd.read_csv('data/nba-2022.csv')
nba

### How many players are on each team?

- We know how to answer this: `.groupby('Team')` with `.count()`.
- **Notice**: `'Team'` becomes the index.

In [ ]:
nba.groupby('Team').count()

### How many players at each position does each team have?

- It seems like grouping would be helpful here, but currently we only know how to group by a single column.
    - Right now, we can count the number of players on each team.
    - Separately, we can count the number of players at each position.

- Here, we want to somehow group by multiple columns. 
    - Specifically, we want the number of players at each position on each team.
    - e.g. the number of centers on the Los Angeles Lakers, the number of power forwards on the Atlanta Hawks.
- There's a way!

In [ ]:
nba.groupby(['Team', 'Position']).count()

### `.groupby` with subgroups

- To make groups within groups, pass a list of column names to `.groupby`:

```py
df.groupby([col_1, col_2, ..., col_k])
```
- Group by `col_1` first.
- Within each group, group by `col_2`, and so on.
- The resulting DataFrame has one row per unique combination of entries in the specified columns.

### Notice the index... 🤔

- This is called a "[MultiIndex](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)".
- We won't worry about the details of MultiIndexes.
- We can use `.reset_index()` to "flatten" our DataFrame back to normal.

In [ ]:
nba.groupby(['Team', 'Position']).count().reset_index()

### Does order matter?

In [ ]:
nba.groupby(['Position', 'Team']).count()

**Answer:** Kind of. The order of the indexes will be different, but the resulting DataFrame will contain the same information.

### Which team has the most centers?

In [ ]:
position_counts = nba.groupby(['Team', 'Position']).count().reset_index()
position_counts

In [ ]:
position_counts[position_counts.get('Position') == 'C'] \
.sort_values('Salary', ascending=False)

In this case, we could've done this without subgroups, too:

In [ ]:
nba[nba.get('Position') == 'C'].groupby('Team').count().sort_values('Position', ascending=False)

### Other aggregation methods work, too!

In [ ]:
nba.groupby(['Team', 'Position']).max().reset_index()

- Remember, the aggregation method is applied to each column individually.
- For instance, Gorgui Dieng does not make a salary of 17,103,448 – his name is just the last (`max`) name alphabetically amongst all centers on the Atlanta Hawks.

In [ ]:
nba[(nba.get('Team') == 'Atlanta Hawks') & (nba.get('Position') == 'C')]

### Example: Sea temperatures 🌊

This dataset contains the sea surface temperature in La Jolla, on many days ranging from August 22, 1916 to May 31, 2019.

In [ ]:
sea_temp = bpd.read_csv('data/sea_temp.csv')
sea_temp

### Discussion Question: Our dataset contains information on 1,234 different months. How do we find the month (e.g. November 1998) with the highest average temperature?

What kind of grouping should we do here?

- A) `sea_temp.groupby('SURFACE_TEMP')`
- B) `sea_temp.groupby('MONTH')`
- C) `sea_temp.groupby(['YEAR', 'MONTH'])`
- D) `sea_temp.groupby(['MONTH', 'DAY'])`
- E) `sea_temp.groupby(['MONTH', 'SURFACE_TEMP'])`

### To answer, go to [menti.com](https://menti.com) and enter the code 9142 6449.

In [ ]:
...

### Plot of yearly average surface temperature 📈

In [ ]:
sea_temp.groupby('YEAR') \
        .mean() \
        .plot(kind='line', y='SURFACE_TEMP');

### Summary: `.groupby` with subgroups

- Pass a list of columns to `.groupby` to make subgroups.
- **Always** use `.reset_index()` after grouping with subgroups, to move the MultiIndex back to the columns.

## Merging 🚗

In [ ]:
phones = bpd.DataFrame().assign(
    Model=['iPhone 13', 'iPhone 13 Pro Max', 'Samsung Galaxy Z Flip', 'Pixel 5a'],
    Price=[799, 1099, 999, 449],
    Screen=[6.1, 6.7, 6.7, 6.3]
)

inventory = bpd.DataFrame().assign(
    Handset=['iPhone 13 Pro Max', 'iPhone 13', 'Pixel 5a', 'iPhone 13'],
    Units=[50, 40, 10, 100],
    Store=['Westfield UTC', 'Westfield UTC', 'Fashion Valley', 'Downtown']
)

In [ ]:
# Phones on the market right now
phones

In [ ]:
# Which phones my stores have in stock in the area
inventory

**Question:** If I sell all of the phones in my inventory, how much will I make in revenue?

### If I sell all of the phones in my inventory, how much will I make in revenue?

In [ ]:
phones.merge(inventory, left_on='Model', right_on='Handset')

### What just happened!? 🤯

In [ ]:
# Click through the presentation that appears
merging_animation()

### `.merge`

- Pick a "left" and "right" DataFrame.
- Choose a column from each to "merge on".
```python
left_df.merge(
    right_df, 
    left_on=left_column_name,
    right_on=right_column_name
)
```
- `left_on` and `right_on` should be column names (they don't have to be the same).
- The resulting DataFrame contains a single row for every match between the two columns.
- Rows in either DataFrame without a match disappear!

### If I sell all of the phones in my inventory, how much will I make in revenue?

In [ ]:
phones

In [ ]:
inventory

In [ ]:
# Notice there's no Samsung Galaxy Z Flip in phones_merged
phones_merged = phones.merge(inventory, left_on='Model', right_on='Handset')
phones_merged

In [ ]:
(phones_merged.get('Price') * phones_merged.get('Units')).sum()

### Shortcut if column names are the same: `on`

In [ ]:
phones

In [ ]:
inventory_relabeled = inventory.assign(Model=inventory.get('Handset')).drop(columns=['Handset'])
inventory_relabeled

In [ ]:
phones.merge(inventory_relabeled, on='Model')

**Notice**: There's only one column containing phone names now.

### Does order matter? 🤔

In [ ]:
phones

In [ ]:
inventory

In [ ]:
inventory.merge(phones, left_on='Handset', right_on='Model')

**Answer:** The order of the rows and columns will be different, but the content will be the same.

### What if we want to "merge on" an index?

Instead of using `left_on` or `right_on`, use `left_index=True` or `right_index=True`.

In [ ]:
phones

In [ ]:
inventory_by_handset = inventory.set_index('Handset')
inventory_by_handset

In [ ]:
phones.merge(inventory_by_handset, left_on='Model', right_index=True)

### Discussion Question Setup

In [ ]:
nice_weather_cities = bpd.DataFrame().assign(
    city=['La Jolla', 'San Diego', 'Austin', 'Los Angeles'],
    state=['California', 'California', 'Texas', 'California'],
    today_high_temp=['79', '83', '87', '87']
    
)

schools = bpd.DataFrame().assign(
    name=['UCSD', 'University of Chicago', 'University of San Diego','Johns Hopkins University', 'UT Austin', 'SDSU', 'UCLA'], 
    city=['La Jolla', 'Chicago', 'San Diego', 'Baltimore', 'Austin', 'San Diego', 'Los Angeles'],
    state=['California', 'Illinois', 'California', 'Maryland', 'Texas', 'California', 'California'],
    graduation_rate=[0.87, 0.94, 0.78, 0.92, 0.81, 0.83, 0.91 ]
)

### Discussion Question

In [ ]:
nice_weather_cities

In [ ]:
schools

**How many rows will be in the output of `nice_weather_cities.merge(schools, on='city')`?**

<center>
<pre>
A. 4      B. 5      C. 6       D. 7       E. 8
</pre>
</center>

### To answer, go to [menti.com](https://menti.com) and enter the code 9142 6449.

### Followup: How many rows will be in the output of `nice_weather_cities.merge(schools, on='state')`?

## Summary

### Summary

- To create groups within a group, pass a list to `.groupby`.
    - The result has one row for every unique combination of elements in the specified columns.
- To combine information from multiple DataFrames, use `.merge`.
    - When using `.merge`, Python searches for a match between a specified column in each DataFrame and combines the rows with a match.
    - No matches = row disappears!
- **Next time**: if-statements and loops.